In [ ]:
import pygame
import random
import sqlite3

# --- Инициализация базы данных ---
def init_db():
    conn = sqlite3.connect("snake_game.db")
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS user_score (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            score INTEGER DEFAULT 0,
            level INTEGER DEFAULT 1,
            FOREIGN KEY (user_id) REFERENCES users(id)
        )
    ''')
    conn.commit()
    conn.close()

def get_or_create_user(username):
    conn = sqlite3.connect("snake_game.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM users WHERE username = ?", (username,))
    result = cursor.fetchone()
    if result:
        user_id = result[0]
    else:
        cursor.execute("INSERT INTO users (username) VALUES (?)", (username,))
        user_id = cursor.lastrowid
        conn.commit()
    cursor.execute("SELECT score, level FROM user_score WHERE user_id = ?", (user_id,))
    score_data = cursor.fetchone()
    if not score_data:
        cursor.execute(
            "INSERT INTO user_score (user_id, score, level) VALUES (?, ?, ?)",
            (user_id, 0, 1)
        )
        conn.commit()
        score_data = (0, 1)
    conn.close()
    return user_id, score_data[0], score_data[1]

def save_score(user_id, score, level):
    conn = sqlite3.connect("snake_game.db")
    cursor = conn.cursor()
    cursor.execute(
        "UPDATE user_score SET score = ?, level = ? WHERE user_id = ?",
        (score, level, user_id)
    )
    conn.commit()
    conn.close()

# --- Перед Pygame: ввод имени и загрузка данных ---
init_db()
username = input("Enter your username: ")
user_id, score, level = get_or_create_user(username)

# --- Инициализация Pygame ---
pygame.init()
WIDTH, HEIGHT = 600, 400
BLOCK_SIZE = 20
WHITE, GREEN, RED, BLACK = (255,255,255), (0,255,0), (255,0,0), (0,0,0)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")
font = pygame.font.SysFont("Arial", 24)
clock = pygame.time.Clock()

# --- Вспомогательные функции рисования ---
def draw_snake(snake):
    for x,y in snake:
        pygame.draw.rect(screen, GREEN, (x, y, BLOCK_SIZE, BLOCK_SIZE))

def draw_food(food):
    x,y = food
    pygame.draw.rect(screen, RED, (x, y, BLOCK_SIZE, BLOCK_SIZE))

def draw_text(text, x, y, color=WHITE):
    img = font.render(text, True, color)
    screen.blit(img, (x,y))

def random_food():
    return [
        random.randint(0, (WIDTH - BLOCK_SIZE)//BLOCK_SIZE) * BLOCK_SIZE,
        random.randint(0, (HEIGHT - BLOCK_SIZE)//BLOCK_SIZE) * BLOCK_SIZE
    ]

# --- Начальное состояние игры ---
center_x, center_y = WIDTH//2, HEIGHT//2
snake = [[center_x, center_y]]
direction = "RIGHT"
food = random_food()
speed = 5  # немного медленнее, чтоб успевать видеть

# Показываем стартовую картинку 2 секунды
screen.fill(BLACK)
draw_snake(snake)
draw_food(food)
draw_text(f"{username} | Score: {score} | Level: {level}", 10, 10)
pygame.display.update()
pygame.time.delay(2000)

# --- Основной цикл ---
running = True
paused = False

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT  and direction!="RIGHT":  direction="LEFT"
            if event.key == pygame.K_RIGHT and direction!="LEFT":   direction="RIGHT"
            if event.key == pygame.K_UP    and direction!="DOWN":   direction="UP"
            if event.key == pygame.K_DOWN  and direction!="UP":     direction="DOWN"
            if event.key == pygame.K_p:
                paused = not paused
                if paused:
                    save_score(user_id, score, level)
                    print("Progress saved!")

    if not paused:
        # двигаем голову
        head = snake[-1].copy()
        if   direction=="LEFT":  head[0] -= BLOCK_SIZE
        elif direction=="RIGHT": head[0] += BLOCK_SIZE
        elif direction=="UP":    head[1] -= BLOCK_SIZE
        elif direction=="DOWN":  head[1] += BLOCK_SIZE

        # проверка столкновений
        if ( head in snake
             or head[0]<0 or head[1]<0
             or head[0]>=WIDTH or head[1]>=HEIGHT ):
            save_score(user_id, score, level)
            screen.fill(BLACK)
            draw_text("GAME OVER! Press any key to exit...", 100, HEIGHT//2)
            pygame.display.update()
            # ждём нажатия или закрытия
            over = True
            while over:
                for e in pygame.event.get():
                    if e.type in (pygame.QUIT, pygame.KEYDOWN):
                        over = False
                        running = False
            break

        snake.append(head)
        # съели еду?
        if head == food:
            score += 10
            if score % 50 == 0:
                level += 1
                speed += 1
            food = random_food()
        else:
            snake.pop(0)

    # отрисовка каждого кадра
    screen.fill(BLACK)
    draw_snake(snake)
    draw_food(food)
    draw_text(f"{username} | Score: {score} | Level: {level}", 10, 10)
    pygame.display.update()
    clock.tick(speed)

pygame.quit()